In [138]:
#jupyter nbextension enable --py gmaps

In [397]:
# Dependencies
from census import Census
from config import (ow_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests, json
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import re

In [398]:
file = "../WeatherPy/weather-data.csv"
weather_data_df = pd.read_csv(file, encoding="utf-8")

weather_data_df.drop('Degrees From Equator', inplace=True, axis=1)
weather_data_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Turan,16.0678,108.2208,32.0,70.0,20.0,5.14
1,Rikitea,-23.1203,-134.9692,26.0,74.0,100.0,7.05
2,Saint-Philippe,-21.3585,55.7679,25.0,69.0,0.0,7.72
3,Karratha,-20.7377,116.8463,33.0,18.0,0.0,6.25
4,Port Elizabeth,-33.9180,25.5701,13.0,100.0,75.0,2.57


In [399]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)


In [400]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data_df[["Latitude", "Longitude"]]

humidity = weather_data_df["Humidity"]

In [401]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(0.0, 30.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weather_data_df["Humidity"].max(),
                                 point_radius = 3)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [405]:
ideal_vacay_spots_df = weather_data_df.loc[(weather_data_df['Temperature'] >= 22) &  
                                        (weather_data_df['Temperature'] <= 25) &
                                        (weather_data_df['Wind Speed'] <= 5) & 
                                        (weather_data_df['Cloudiness'] == 0)
                                        ].reset_index(drop=True)

ideal_vacay_spots_df[['Hotel', 'Hotel Rating', 'Country']] = ''
ideal_vacay_spots_df


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel,Hotel Rating,Country
0,Cabedelo,-6.9811,-34.8339,24.0,94.0,0.0,1.54,,,
1,Gibara,21.1097,-76.1317,24.0,88.0,0.0,3.09,,,
2,San Ignacio,17.1561,-89.0714,24.0,78.0,0.0,2.06,,,
3,Sakakah,29.9697,40.2064,25.0,13.0,0.0,3.60,,,
4,Terenos,-20.4422,-54.8603,23.0,56.0,0.0,4.12,,,
5,Pitimbu,-7.4706,-34.8086,24.0,94.0,0.0,1.54,,,


In [406]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

# 6. Bonus: Find the names and addresses of the top five restaurants in your home city.
#    Hint: Read about "Text Search Results"
# (https://developers.google.com/places/web-service/search#TextSearchRequests)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in ideal_vacay_spots_df.iterrows():
   # print (f"{row[1]},{row[3]}")
    params = {
    "location": f"{row[1]},{row[2]}",
    "radius": 5000,
    "type": "lodging",
    "keyword" : "hotel",
    "language" : "en",
    "key": gkey
    }
    
    best_hotel_rating = 0
    best_hotels = requests.get(base_url, params=params).json()
    #print(best_hotels)
    for hotel in best_hotels['results']:
        #print (index, hotel)
        #print (hotel['rating'])
        try:
            if hotel['rating'] > best_hotel_rating:
                ideal_vacay_spots_df.at[index, 'Hotel'] = hotel['name']
                ideal_vacay_spots_df.at[index, 'Hotel Rating'] = hotel['rating']
                country, = re.findall('\s.*,\s(.*?)$', hotel['plus_code']['compound_code'])
                ideal_vacay_spots_df.at[index, 'Country'] = country
                #ideal_vacay_spots_df.at[index, 'Place ID'] = hotel['place_id']
        except (KeyError, ValueError):
                ideal_vacay_spots_df.at[index, 'Country'] = ''

    

In [407]:

ideal_vacay_spots_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel,Hotel Rating,Country
0,Cabedelo,-6.9811,-34.8339,24.0,94.0,0.0,1.54,Hotel Almagre,4.4,Brazil
1,Gibara,21.1097,-76.1317,24.0,88.0,0.0,3.09,Sol y Mar,4.6,Cuba
2,San Ignacio,17.1561,-89.0714,24.0,78.0,0.0,2.06,Cohune Palms River Cabanas,4.5,Belize
3,Sakakah,29.9697,40.2064,25.0,13.0,0.0,3.60,أجنحة رحاب الضيافة,3.3,
4,Terenos,-20.4422,-54.8603,23.0,56.0,0.0,4.12,Hotel Renascer,4.7,Brazil
5,Pitimbu,-7.4706,-34.8086,24.0,94.0,0.0,1.54,Pousada O Nativo,4.2,Brazil


In [418]:
# Create a map using state centroid coordinates to set markers
marker_locations = ideal_vacay_spots_df[['Latitude', 'Longitude']]

info = []
for index, row in ideal_vacay_spots_df.iterrows():
    text = f"<dl><dt>Hotel</dt><dd>{row[7]}</dd><dt>City</dt><dd>{row[0]}</dd><dt>Country</dt><dd>{row[9]}</dd>"
    info.append(text)

# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(marker_locations, info_box_content= info)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))